In [31]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [5]:
def train_network():
    """ Train a Neural Network to generate music """
#     notes = get_notes()

#     # get amount of pitch names
#     n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [3]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


In [4]:
get_notes()

Parsing midi_songs/d-celebratethelove.mid
Parsing midi_songs/d-anything.mid
Parsing midi_songs/d-closetoyou.mid
Parsing midi_songs/d-babybaby.mid
Parsing midi_songs/d-anotherday.mid
Parsing midi_songs/d-becauseimlovingyou.mid
Parsing midi_songs/d-anothernight.mid
Parsing midi_songs/borntobe_d.mid
Parsing midi_songs/blkisblk_d.mid
Parsing midi_songs/d-100purelove.mid
Parsing midi_songs/d-automaticlover.mid
Parsing midi_songs/d-dontcryformeargentina.mid
Parsing midi_songs/d-badboy.mid
Parsing midi_songs/d-beautifullife.mid
Parsing midi_songs/borntobewild2.mid
Parsing midi_songs/d-dadip.mid
Parsing midi_songs/d-cometakemyhand.mid
Parsing midi_songs/d-bemylover.mid
Parsing midi_songs/d-cocojamboo.mid
Parsing midi_songs/d-dontgoaway.mid
Parsing midi_songs/d-childrenofthenight.mid
Parsing midi_songs/d-desire.mid
Parsing midi_songs/celebrat_d.mid
Parsing midi_songs/d-123trainwithme.mid
Parsing midi_songs/d-allthatshewants.mid
Parsing midi_songs/d-captainjack.mid
Parsing midi_songs/blowthe_d.m

['B4',
 'B1',
 'B4',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B2',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',

In [6]:
notes = get_notes()
notes

Parsing midi_songs/d-celebratethelove.mid
Parsing midi_songs/d-anything.mid
Parsing midi_songs/d-closetoyou.mid
Parsing midi_songs/d-babybaby.mid
Parsing midi_songs/d-anotherday.mid
Parsing midi_songs/d-becauseimlovingyou.mid
Parsing midi_songs/d-anothernight.mid
Parsing midi_songs/borntobe_d.mid
Parsing midi_songs/blkisblk_d.mid
Parsing midi_songs/d-100purelove.mid
Parsing midi_songs/d-automaticlover.mid
Parsing midi_songs/d-dontcryformeargentina.mid
Parsing midi_songs/d-badboy.mid
Parsing midi_songs/d-beautifullife.mid
Parsing midi_songs/borntobewild2.mid
Parsing midi_songs/d-dadip.mid
Parsing midi_songs/d-cometakemyhand.mid
Parsing midi_songs/d-bemylover.mid
Parsing midi_songs/d-cocojamboo.mid
Parsing midi_songs/d-dontgoaway.mid
Parsing midi_songs/d-childrenofthenight.mid
Parsing midi_songs/d-desire.mid
Parsing midi_songs/celebrat_d.mid
Parsing midi_songs/d-123trainwithme.mid
Parsing midi_songs/d-allthatshewants.mid
Parsing midi_songs/d-captainjack.mid
Parsing midi_songs/blowthe_d.m

['B4',
 'B1',
 'B4',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B1',
 'B1',
 'B1',
 'B1',
 'B1',
 'D2',
 'B2',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',
 'B1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'G0',
 'G1',
 'G1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'A0',
 'A1',
 'A1',
 'B0',
 'B1',
 'B1',
 'B0',
 'B1',

In [7]:
n_vocab = len(set(notes))

In [11]:
n_vocab

382

In [8]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 30

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [9]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [10]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=30, batch_size=64, callbacks=callbacks_list)

In [13]:
train_network()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
68075/68075 [==============================] - 1048s 15ms/step - loss: 4.4730
Epoch 2/30
68075/68075 [==============================] - 991s 15ms/step - loss: 4.2867
Epoch 3/30
68075/68075 [==============================] - 956s 14ms/step - loss: 4.2145
Epoch 4/30
68075/68075 [==============================] - 957s 14ms/step - loss: 4.1138
Epoch 5/30
68075/68075 [==============================] - 1004s 15ms/step - loss: 3.8736
Epoch 6/30
68075/68075 [==============================] - 1034s 15ms/step - loss: 3.5210
Epoch 7/30
68075/68075 [==============================] - 1020s 15ms/step - loss: 3.1539
Epoch 8/30
68075/68075 [==============================] - 1052s 15ms/step - loss: 2.7338
Epoch 9/30
68075/68075 [==============================] 

In [25]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [26]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map back from integers to notes
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [27]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/Users/leesurkis/Documents/GitHub/Music_Files/Generating_Techno_Music/weights-improvement-30-0.4634-bigger.hdf5')

    return model

In [28]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction) # numpy array of predictions
        result = int_to_note[index] # indexing the note with the highest probability
        prediction_output.append(result) # that note is the prediction output

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [29]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.SnareDrum()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.SnareDrum()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output2.mid')

In [30]:
generate()